In [1]:
import numpy as np
import polars as pl

In [2]:
file = pl.read_csv("rawData.csv")
file.shape

(19703, 10)

In [52]:
file = file.drop('cityName')
file.head(1)

temp,feelsLike,pressure,humidity,clouds,windSpeed,windDir,gustSpeed,result
f64,f64,f64,f64,f64,f64,f64,f64,str
25.3,25.8,982.0,74.0,25.0,3.04,242.0,4.0,"""Scattered clou…"


In [53]:
file.null_count()

temp,feelsLike,pressure,humidity,clouds,windSpeed,windDir,gustSpeed,result
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [54]:
resUnique = np.array(file['result'].unique())
resUnique

array(['Heavy rain', 'Fog', 'Overcast clouds', 'Broken clouds',
       'Thunderstorm with heavy rain', 'Moderate rain',
       'Scattered clouds', 'Haze', 'Light rain', 'Few clouds',
       'Clear Sky'], dtype='<U28')

In [55]:
file = file.with_columns(
  pl.when(pl.col('result') == 'Scattered clouds').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Clear Sky').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Few clouds').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Haze').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Overcast clouds').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Fog').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Broken clouds').then(pl.lit('no rain'))
  .when(pl.col('result') == 'Moderate rain').then(pl.lit('rain'))
  .when(pl.col('result') == 'Thunderstorm with heavy rain').then(pl.lit('rain'))
  .when(pl.col('result') == 'Light rain').then(pl.lit('rain'))
  .when(pl.col('result') == 'Heavy rain').then(pl.lit('rain')).alias("result")
)

file.head(3)

temp,feelsLike,pressure,humidity,clouds,windSpeed,windDir,gustSpeed,result
f64,f64,f64,f64,f64,f64,f64,f64,str
25.3,25.8,982.0,74.0,25.0,3.04,242.0,4.0,"""no rain"""
25.3,25.9,982.0,74.0,25.0,2.38,232.0,3.0,"""no rain"""
26.0,26.5,983.0,71.0,25.0,3.02,241.0,3.02,"""no rain"""


In [56]:
file = file.with_columns(
  pl.when(pl.col('result') == 'no rain').then(0)
  .when(pl.col('result') == 'rain').then(1).alias('result')
)25.8	982.0	74.0	25.0	3.25.8	982.0	74.0	25.0	3.04	242.0	4.0	"no rain"
25.3	25.9	982.0	74.0	25.0	2.38	232.0	3.0	"no rain"
26.0	26.5	983.0	04	242.0	4.0	"no rain"
25.3	25.9	982.0	74.0	25.0	2.38	232.0	3.0	"no rain"
26.0	26.5	983.0	
file.head(3)

temp,feelsLike,pressure,humidity,clouds,windSpeed,windDir,gustSpeed,result
f64,f64,f64,f64,f64,f64,f64,f64,i32
25.3,25.8,982.0,74.0,25.0,3.04,242.0,4.0,0
25.3,25.9,982.0,74.0,25.0,2.38,232.0,3.0,0
26.0,26.5,983.0,71.0,25.0,3.02,241.0,3.02,0


In [57]:
file['result'].unique()

result
i32
0
1


In [58]:
rainData = file.filter(pl.col('result') == 0)
norainData = file.filter(pl.col('result') == 1)

writeableDf = pl.DataFrame()
writeableDf = pl.concat([writeableDf, norainData])
writeableDf = pl.concat([writeableDf, rainData[0: 2226]])
writeableDf

file.head(3)

temp,feelsLike,pressure,humidity,clouds,windSpeed,windDir,gustSpeed,result
f64,f64,f64,f64,f64,f64,f64,f64,i32
25.3,25.8,982.0,74.0,25.0,3.04,242.0,4.0,0
25.3,25.9,982.0,74.0,25.0,2.38,232.0,3.0,0
26.0,26.5,983.0,71.0,25.0,3.02,241.0,3.02,0


In [59]:
writeableDf.write_csv("preprocessedData.csv")